In [ ]:
import numpy as np
import urllib.request
from PIL import Image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from google.colab import drive
import os
import pandas as pd
import uuid
from concurrent.futures import ThreadPoolExecutor

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load pre-trained InceptionV3 model
model = InceptionV3(weights='imagenet')

# Function to generate tags for an image given its URL
def generate_tags_from_image_url(image_url):
    try:
        # Load and preprocess the image from URL
        with urllib.request.urlopen(image_url) as url:
            img = Image.open(url)
            img = img.resize((299, 299))  # Resize the image to match the input size of InceptionV3
            img_array = np.expand_dims(np.array(img), axis=0)
            img_array = preprocess_input(img_array)  # Preprocess the image for InceptionV3

        # Make predictions on the image
        predictions = model.predict(img_array)

        # Decode predictions to get class labels
        decoded_predictions = decode_predictions(predictions, top=6)[0]

        # Extract tags from decoded predictions
        tags = [{label[1].replace('_', ' '): label[2]} for label in decoded_predictions]

        return tags
    except Exception as e:
        print(f"Error generating tags for image: {e}")
        return None

96112376/96112376 [==============================] - 0s 0us/step


In [ ]:
def generate_uuid():
    return str(uuid.uuid4())

In [ ]:
# Example usage
image_url = "https://images.unsplash.com/photo-1587135941948-670b381f08ce?q=80&w=2070"
generated_tags = generate_tags_from_image_url(image_url)

if generated_tags:
    print("Generated tags for the image:", generated_tags)
else:
    print("Failed to generate tags for the image.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[('n03788195', 'mosque', 0.9526032), ('n03877845', 'palace', 0.0024458563), ('n03220513', 'dome', 0.0021314113), ('n03447447', 'gondola', 0.0016001905), ('n04346328', 'stupa', 0.000601429), ('n03998194', 'prayer_rug', 0.00043926458)]
Generated tags for the image: [{'mosque': 0.9526032}, {'palace': 0.0024458563}, {'dome': 0.0021314113}, {'gondola': 0.0016001905}, {'stupa': 0.000601429}, {'prayer rug': 0.00043926458}]


In [ ]:
directory = "/content/drive/MyDrive/MOS_2/Data_v3"
tagged_directory = "/content/drive/MyDrive/MOS_2/Tagged_v3"
files = os.listdir(directory)

In [ ]:
csv_files = [directory + "/" + file for file in files if file.endswith('.csv')]

In [ ]:
len(csv_files)

229

In [ ]:
print(csv_files[0])

/content/drive/MyDrive/MOS_2/Data_v3/africans_image_data.csv


In [ ]:
def process_csv(file):
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    df['image_tags'] = df['URL'].apply(generate_tags_from_image_url)
    df['Unique_ID'] = [generate_uuid() for _ in range(len(df))]  # Generate unique IDs
    filename = os.path.basename(file)
    processed_file_path = f"{tagged_directory}/{filename}"
    df.to_csv(processed_file_path, index=False)

In [ ]:
process_csv(directory + "/funny cats_image_data.csv")

In [ ]:
with ThreadPoolExecutor() as executor:
    executor.map(process_csv, csv_files)